<p align=center><img src=https://d31uz8lwfmyn8g.cloudfront.net/Assets/logo-henry-white-lg.png><p>

# <h1 align=center> **PROYECTO INDIVIDUAL Nº1** </h1>
# <h1 align=center>¡Bienvenidos a mi primer proyecto individual de la etapa de labs! En esta ocasión, realicé el trabajo que desarrolla un ***MLOps Engineer***. </h1> 

 

In [1]:
#Importamos librerias a utilizar
import json
import pandas as pd
import ast
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pyarrow.parquet as pq
import pyarrow as pa


### Procesamiento del archivo Json user_reviews

In [33]:
#Leemos el archivo .Json y lo convertimos a Df con pandas
filas = list()
with open("australian_user_reviews.json","r", encoding="Latin-1") as archivo:
    for linea in archivo.readlines():
        filas.append(ast.literal_eval(linea))
df_reviews=pd.DataFrame(filas)

In [34]:
df_reviews.head(2)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."


- Obervamos la columna reviews dentro de una lista de diccionarios, vamos a procesarla

In [35]:
df_reviews_explode= df_reviews.explode('reviews').reset_index()#Explode para separar los reviews
df_reviews_f= pd.concat([df_reviews_explode.drop(columns='reviews'),df_reviews_explode['reviews'].apply(pd.Series)],axis=1)#Concatenamos el df con los reviews

In [36]:
df_reviews_f.head()#Visualizamos el df

,index,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,0
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN
3,1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN
4,1,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN


In [37]:
df_reviews_f= df_reviews_f.drop(df_reviews_f.columns[-1],axis=1) #Eliminamos la ultima columna que se creó al hacer el explode

In [38]:
df_reviews_f= df_reviews_f.drop(columns=['funny','last_edited','user_url','index'])#Eliminamos columnas que no nos sirven

In [39]:
df_reviews_f#Visualizamos el df

,user_id,posted,item_id,helpful,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,Posted July 8.,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,Posted July 3.,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,Posted July 20.,730,No ratings yet,True,:D


- Vamos a procesar la columna review, aplicando SentimentIntensityAnalyzer para obtener el analisis de sentimiento de cada review, y asi poder clasificarlos en positivos, negativos y neutros

In [40]:
analyzer = SentimentIntensityAnalyzer()              #Inicializamos el analizador de sentimientos
def analyze_sentiment(review):
    if isinstance(review, str) and review.strip():   #Si la revisión es una cadena no vacía
        sentiment = analyzer.polarity_scores(review) #Obtenemos el sentimiento
        if sentiment['compound'] >= 0.05:            #Si el sentimiento es mayor a 0.05
            return 2                                 #2 si es positivo 
        elif sentiment['compound'] <= -0.05:         #Si el sentimiento es menor a -0.05
            return 0                                 #0 si es negativo
        else:
            return 1                                 #si es neutral
    else:
        return 1                                     #1 si no hay revisión o está vacío
    
    

# Aplicar la función de análisis de sentimiento a la columna 'review'
df_reviews_f['sentiment_analysis'] = df_reviews_f['review'].apply(analyze_sentiment)

In [41]:
# El DataFrame 'df' ahora tendrá una nueva columna 'sentiment_analysis' con valores 0, 1 o 2 según el análisis de sentimiento o 1 si no hay revisión.
df_reviews_f['sentiment_analysis'].value_counts()

sentiment_analysis
2    37741
1    12074
0     9518
Name: count, dtype: int64

In [44]:
df_reviews_f['posted_year'] = df_reviews_f['posted'].str.extract(r'(\d{4})')#Extraemos el año de la columna posted

In [49]:
df_reviews_f.drop(columns='posted',inplace=True) #Eliminamos la columna posted

In [63]:
df_reviews_f.isna().sum()#Verificamos si hay valores nulos

user_id                   0
item_id                  28
helpful                  28
recommend                28
review                   28
sentiment_analysis        0
posted_year           10147
dtype: int64

In [66]:
df_reviews_f.dropna(subset='posted_year',inplace=True) #Eliminamos los valores nulos de la columna posted_year

In [71]:
df_reviews_f.drop(columns='review',inplace=True) #Eliminamos la columna review, ya que tenemos la de analisis de sentimientos

In [73]:
df_final= df_reviews_f.to_csv('reviews.csv',index=False) # pasamos a Csv 

### Procesamiento del archivo Json users_items

In [2]:
#Leemos el archivo .Json y lo convertimos a Df con pandas
filas2 = list()
with open("australian_users_items.json","r", encoding="Latin-1") as archivo:
    for linea in archivo.readlines():
        filas2.append(ast.literal_eval(linea))
df_items= pd.DataFrame(filas2)

In [3]:
df_items= df_items.explode('items')#Explode para separar items, para separar los valores

In [4]:
df_items.reset_index(inplace=True)#Reseteamos el index para que este en orden

In [6]:
df_items.drop(columns=['index'],inplace=True) #Eliminamos la columna index que se creo al hacer el reset

In [8]:
df_items.drop(columns=['user_url','steam_id'],inplace=True) #Eliminamos las columnas que no nos sirven por el momento

In [10]:
df_items_c = pd.concat([df_items, pd.json_normalize(df_items['items'])], axis=1) #Concatenamos el df con los items

In [12]:
df_items_c.drop(columns=['items'],inplace=True)#Eliminamos la columna items

In [20]:
df_items_final= df_items_c.to_csv('items.csv',index=False) # pasamos a Csv 

### Procesamiento del archivo Json steam_games

In [49]:
#Leemos el archivo .Json y lo convertimos a Df con pandas
filas = []
with open("output_steam_games.json", "r", encoding="Latin-1") as archivo:
    for linea in archivo:
        try:
            objeto_json = json.loads(linea)
            filas.append(objeto_json)
        except json.JSONDecodeError:
            print(f"Error de formato JSON en la línea: {linea}")

df_games = pd.DataFrame(filas)
df_games.tail()#Visualizamos el df ultimas 5 filas

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"
120444,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550,NaN


- Una primera impresion sobre el df, observamos que hay columnas que no nos sirven, está la columna genres que tiene listas, release_date hay que extraer el año. Veremos poco a poco el seguimiento

In [50]:
df_games['release_year'] = df_games['release_date'].str.extract(r'(\d{4})') #Extraemos el año de la columna release_date

In [52]:
df_games.drop(columns=['early_access','reviews_url','specs','url','tags','title','release_date'],inplace=True) #Eliminamos las columnas que no nos sirven
df_games.dropna(subset=['id'], inplace=True) #Eliminamos los valores nulos de la columna id

In [53]:
df_games

,publisher,genres,app_name,price,id,developer,release_year
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,4.99,761140,Kotoshiro,2018
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Free To Play,643980,Secret Level SRL,2018
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Free to Play,670290,Poolians.com,2017
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,0.99,767400,彼岸领域,2017
88314,NaN,NaN,Log Challenge,2.99,773570,NaN,NaN
...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,1.99,773640,"Nikita ""Ghost_RUS""",2018
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,4.99,733530,Sacada,2018
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,1.99,610660,Laush Dmitriy Sergeevich,2018
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,4.99,658870,"xropi,stev3ns",2017


- Obervamos que los valores de la columna genres, están dentro de listas, por lo que debemos extraerlos y aplicar un dummies para poder trabajar con ellos 

In [54]:
df_games['genres'] = df_games['genres'].fillna('[]')                   #Rellenamos los valores nulos de la columna genres con []
df_games['genres'] = df_games['genres'].apply(lambda x: ', '.join(x))  # Convertir la lista de géneros a una cadena separada por comas


dummy_genres = df_games['genres'].str.get_dummies(', ')                # Crear variables dummies para cada género


df_steam_games = pd.concat([df_games, dummy_genres], axis=1)           # Concatenar el DataFrame original con las variables dummies

In [55]:
df_steam_games

,publisher,genres,app_name,price,id,developer,release_year,Accounting,Action,Adventure,...,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,[,]
88310,Kotoshiro,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,4.99,761140,Kotoshiro,2018,0,1,0,...,0,1,0,0,1,0,0,0,0,0
88311,"Making Fun, Inc.","Free to Play, Indie, RPG, Strategy",Ironbound,Free To Play,643980,Secret Level SRL,2018,0,0,0,...,0,0,0,0,1,0,0,0,0,0
88312,Poolians.com,"Casual, Free to Play, Indie, Simulation, Sports",Real Pool 3D - Poolians,Free to Play,670290,Poolians.com,2017,0,0,0,...,0,1,0,1,0,0,0,0,0,0
88313,彼岸领域,"Action, Adventure, Casual",弹炸人2222,0.99,767400,彼岸领域,2017,0,1,1,...,0,0,0,0,0,0,0,0,0,0
88314,NaN,"[, ]",Log Challenge,2.99,773570,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"Casual, Indie, Simulation, Strategy",Colony On Mars,1.99,773640,"Nikita ""Ghost_RUS""",2018,0,0,0,...,0,1,0,0,1,0,0,0,0,0
120441,Sacada,"Casual, Indie, Strategy",LOGistICAL: South Africa,4.99,733530,Sacada,2018,0,0,0,...,0,0,0,0,1,0,0,0,0,0
120442,Laush Studio,"Indie, Racing, Simulation",Russian Roads,1.99,610660,Laush Dmitriy Sergeevich,2018,0,0,0,...,1,1,0,0,0,0,0,0,0,0
120443,SIXNAILS,"Casual, Indie",EXIT 2 - Directions,4.99,658870,"xropi,stev3ns",2017,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
df_steam_games = df_steam_games.drop(df_steam_games.columns[-2:], axis=1) #quedaron 2 columnas-> [   , -> ] ... al final del df
df_steam_games.reset_index(drop=True, inplace=True)
df_steam_games

,publisher,genres,app_name,price,id,developer,release_year,Accounting,Action,Adventure,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,Kotoshiro,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,4.99,761140,Kotoshiro,2018,0,1,0,...,0,0,0,1,0,0,1,0,0,0
1,"Making Fun, Inc.","Free to Play, Indie, RPG, Strategy",Ironbound,Free To Play,643980,Secret Level SRL,2018,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,Poolians.com,"Casual, Free to Play, Indie, Simulation, Sports",Real Pool 3D - Poolians,Free to Play,670290,Poolians.com,2017,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,彼岸领域,"Action, Adventure, Casual",弹炸人2222,0.99,767400,彼岸领域,2017,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,NaN,"[, ]",Log Challenge,2.99,773570,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32128,Ghost_RUS Games,"Casual, Indie, Simulation, Strategy",Colony On Mars,1.99,773640,"Nikita ""Ghost_RUS""",2018,0,0,0,...,0,0,0,1,0,0,1,0,0,0
32129,Sacada,"Casual, Indie, Strategy",LOGistICAL: South Africa,4.99,733530,Sacada,2018,0,0,0,...,0,0,0,0,0,0,1,0,0,0
32130,Laush Studio,"Indie, Racing, Simulation",Russian Roads,1.99,610660,Laush Dmitriy Sergeevich,2018,0,0,0,...,0,0,1,1,0,0,0,0,0,0
32131,SIXNAILS,"Casual, Indie",EXIT 2 - Directions,4.99,658870,"xropi,stev3ns",2017,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
df_steam_games.drop(columns=['genres','publisher'],inplace=True) #Eliminamos la columna genres ya que tenemos las dummies y publisher ya que no nos sirve

In [58]:
df_steam_games.isna().sum()

app_name                        1
price                        1377
id                              0
developer                    3298
release_year                 2167
Accounting                      0
Action                          0
Adventure                       0
Animation &amp; Modeling        0
Audio Production                0
Casual                          0
Design &amp; Illustration       0
Early Access                    0
Education                       0
Free to Play                    0
Indie                           0
Massively Multiplayer           0
Photo Editing                   0
RPG                             0
Racing                          0
Simulation                      0
Software Training               0
Sports                          0
Strategy                        0
Utilities                       0
Video Production                0
Web Publishing                  0
dtype: int64

In [59]:
df_steam_games.fillna('sin_datos', inplace=True)                    #Rellenamos los valores nulos con sin_datos
df_steam_games['price'].replace('sin_datos', 0, inplace=True)       #Reemplazamos los valores sin_datos por 0 de la columna price
df_steam_games['release_year'].replace('sin_datos', 0, inplace=True)#Reemplazamos los valores sin_datos por 0 de la columna release_year

In [60]:
df_steam_games.isna().sum()

app_name                     0
price                        0
id                           0
developer                    0
release_year                 0
Accounting                   0
Action                       0
Adventure                    0
Animation &amp; Modeling     0
Audio Production             0
Casual                       0
Design &amp; Illustration    0
Early Access                 0
Education                    0
Free to Play                 0
Indie                        0
Massively Multiplayer        0
Photo Editing                0
RPG                          0
Racing                       0
Simulation                   0
Software Training            0
Sports                       0
Strategy                     0
Utilities                    0
Video Production             0
Web Publishing               0
dtype: int64

In [61]:
df_steam_games.rename(columns={'id':'item_id'},inplace=True) #Renombramos la columna id a item_id

- Vamos a observar la columna precio, los valores que tienen string que nos impiden convertir a float

In [62]:
var_strings = df_steam_games[df_steam_games['price'].apply(lambda x: isinstance(x, str))]
print(var_strings['price'].value_counts())

price
Free                             905
Free to Play                     520
Free To Play                     462
Free Mod                           4
Free Demo                          3
Play Now                           2
Third-party                        2
Play for Free!                     2
Play WARMACHINE: Tactics Demo      1
Install Theme                      1
Install Now                        1
Free HITMAN™ Holiday Pack          1
Play the Demo                      1
Starting at $499.00                1
Starting at $449.00                1
Free to Try                        1
Free Movie                         1
Free to Use                        1
Name: count, dtype: int64


- Vamos a optar por reemplzar dichos valores, por 0

In [63]:
df_steam_games.loc[df_steam_games['price'].apply(lambda x: isinstance(x, str)), 'price'] = 0 

In [64]:
df_steam_games['price'] = df_steam_games['price'].astype(float) #Convertimos la columna price a float

In [65]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32133 entries, 0 to 32132
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   app_name                   32133 non-null  object 
 1   price                      32133 non-null  float64
 2   item_id                    32133 non-null  object 
 3   developer                  32133 non-null  object 
 4   release_year               32133 non-null  object 
 5   Accounting                 32133 non-null  int64  
 6   Action                     32133 non-null  int64  
 7   Adventure                  32133 non-null  int64  
 8   Animation &amp; Modeling   32133 non-null  int64  
 9   Audio Production           32133 non-null  int64  
 10  Casual                     32133 non-null  int64  
 11  Design &amp; Illustration  32133 non-null  int64  
 12  Early Access               32133 non-null  int64  
 13  Education                  32133 non-null  int

In [ ]:
final= df_steam_games.to_csv('steam_games2.csv', index=False) # pasamos a Csv

- Vamos a tomar los 3 archivos csv y pasarlos a parquet, por cuestiones de almacenamiento

In [4]:
df_games= pd.read_csv('./data/steam_games2.csv') #Leemos el archivo steam_games2.csv
df_items= pd.read_csv('./data/items.csv') #Leemos el archivo items.csv
df_reviews= pd.read_csv('./data/reviews.csv') #Leemos el archivo reviews.csv

In [13]:
table= pa.Table.from_pandas(df_games) #Creamos la tabla de steam_games2.csv
pq.write_table(table, 'steam_games2.parquet') #Pasamos a parquet

In [14]:
table=pq.read_table('steam_games2.parquet') #Leemos la tabla de steam_games2.parquet
games_parquet= table.to_pandas() #Pasamos a pandas
games_parquet

,app_name,price,item_id,developer,release_year,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,Lost Summoner Kitty,4.99,761140,Kotoshiro,2018,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1,Ironbound,0.00,643980,Secret Level SRL,2018,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,Real Pool 3D - Poolians,0.00,670290,Poolians.com,2017,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,弹炸人2222,0.99,767400,彼岸领域,2017,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Log Challenge,2.99,773570,sin_datos,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32128,Colony On Mars,1.99,773640,"Nikita ""Ghost_RUS""",2018,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
32129,LOGistICAL: South Africa,4.99,733530,Sacada,2018,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
32130,Russian Roads,1.99,610660,Laush Dmitriy Sergeevich,2018,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
32131,EXIT 2 - Directions,4.99,658870,"xropi,stev3ns",2017,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Lee el archivo Parquet en un DataFrame de pandas
df_parquet = pd.read_parquet('steam_games2.parquet')


In [17]:
table2= pa.Table.from_pandas(df_items) #Creamos la tabla de steam_games2.csv
pq.write_table(table2, 'user_items.parquet') #Pasamos a parquet

In [18]:
table2=pq.read_table('user_items.parquet') #Leemos la tabla de steam_games2.parquet
items_parquet= table2.to_pandas() #Pasamos a pandas
items_parquet

,user_id,items_count,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,10.0,Counter-Strike,6.0,0.0
1,76561197970982479,277,20.0,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,30.0,Day of Defeat,7.0,0.0
3,76561197970982479,277,40.0,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,50.0,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...
5170010,76561198329548331,7,373330.0,All Is Dust,0.0,0.0
5170011,76561198329548331,7,388490.0,One Way To Die: Steam Edition,3.0,3.0
5170012,76561198329548331,7,521570.0,You Have 10 Seconds 2,4.0,4.0
5170013,76561198329548331,7,519140.0,Minds Eyes,3.0,3.0


In [19]:
table3= pa.Table.from_pandas(df_reviews) #Creamos la tabla de steam_games2.csv
pq.write_table(table3, 'user_reviews.parquet') #Pasamos a parquet

In [20]:
table3=pq.read_table('user_reviews.parquet') #Leemos la tabla de steam_games2.parquet
reviews_parquet= table3.to_pandas() #Pasamos a pandas
reviews_parquet

,user_id,item_id,helpful,recommend,sentiment_analysis,posted_year
0,76561197970982479,1250,No ratings yet,True,2,2011
1,76561197970982479,22200,No ratings yet,True,2,2011
2,76561197970982479,43110,No ratings yet,True,2,2011
3,js41637,251610,15 of 20 people (75%) found this review helpful,True,2,2014
4,js41637,227300,0 of 1 people (0%) found this review helpful,True,2,2013
...,...,...,...,...,...,...
49181,wayfeng,730,1 of 1 people (100%) found this review helpful,True,1,2015
49182,76561198251004808,253980,No ratings yet,True,2,2015
49183,72947282842,730,No ratings yet,True,0,2015
49184,ApxLGhost,730,No ratings yet,True,2,2015


In [4]:
games= pd.read_parquet('./data/steam_games2.parquet') 

In [9]:
games = games.drop(games[games['developer'] == 'sin_datos'].index)

In [10]:
games['developer'].value_counts()

developer
Ubisoft - San Francisco       1259
SmiteWorks USA, LLC            813
Dovetail Games                 253
KOEI TECMO GAMES CO., LTD.     232
Paradox Development Studio     156
                              ... 
Royal9Games,RoyalJL              1
Salus Games                      1
Class of  2016-2017              1
Pirotexnik                       1
Bidoniera Games                  1
Name: count, Length: 10992, dtype: int64

In [11]:
table= pa.Table.from_pandas(games) #Creamos la tabla de steam_games2.csv
pq.write_table(table, 'steam_games22.parquet') #Pasamos a parquet

In [12]:
table=pq.read_table('steam_games22.parquet') #Leemos la tabla de steam_games2.parquet
games_parquet= table.to_pandas() #Pasamos a pandas
games_parquet

,app_name,price,item_id,developer,release_year,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,Lost Summoner Kitty,4.99,761140,Kotoshiro,2018,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1,Ironbound,0.00,643980,Secret Level SRL,2018,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,Real Pool 3D - Poolians,0.00,670290,Poolians.com,2017,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,弹炸人2222,0.99,767400,彼岸领域,2017,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Battle Royale Trainer,3.99,772540,Trickjump Games Ltd,2018,0,1,1,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32127,Kebab it Up!,1.99,745400,Bidoniera Games,2018,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
32128,Colony On Mars,1.99,773640,"Nikita ""Ghost_RUS""",2018,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
32129,LOGistICAL: South Africa,4.99,733530,Sacada,2018,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
32130,Russian Roads,1.99,610660,Laush Dmitriy Sergeevich,2018,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
